<a href="https://colab.research.google.com/github/saurabhsangwan/music-genre-classification/blob/main/RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)

In [ ]:
%cd drive/MyDrive/IDS

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import plotly.figure_factory as ff
import seaborn as sns
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import plotly.figure_factory as ff
from sklearn.feature_selection import SelectKBest, f_classif

In [ ]:
data = pd.read_csv("music-genre-classification/data/merged_data.csv")
data.head()

In [ ]:
data= data.drop(columns=['Unnamed: 0'])

In [ ]:
data.count()

In [ ]:
data.head()

In [ ]:
#explore data
print(f'Number of variables - {data.shape[1]}\n')
print(f'Data Types for each variable - \n{data.dtypes}\n')
print(f'Number of variables for each data type - \n{data.dtypes.value_counts()}')

In [ ]:
#search missing data
#Counting Null values for each column
null_count = data.isnull().sum()
null_count = null_count[null_count>0]

#Printing null counts for columns that have null count >0
print(f'Null count for Columns - \n{null_count}\n\n')

#Printing % of null counts for columns that have null count>0
for column, null_co in null_count.iteritems():
    print(f'Percentage of missing values for Column {column} - {null_co*100/data.shape[0]:.2f} %')
    if (null_co*100/data.shape[0]) > 0:
      col = column 
      data[data.isnull().any(axis=1)][[col]]

In [ ]:
data = data.dropna()

In [ ]:
#Counting Null values for each column
null_count = data.isnull().sum()
null_count = null_count[null_count>0]
print(null_count)

In [ ]:
#Checking boolean False counts for each column, an empty/blank string or integer 0 will result in a False value
#for boolean type
bool_counts = data.astype(bool).sum(axis=0)
for col, val in bool_counts.iteritems():
    print(f'{col} Percentage Boolean False Counts {(data.shape[0]-val)*100/data.shape[0]:.2f}')

In [ ]:
sns.countplot(x = 'genre', data = data)

Partition data

In [ ]:
data.count()

In [ ]:
data_new = data.drop(columns=['genre','title', 'track_id'])

In [ ]:
#classification data splitting 
X_train,X_test, Y_train, Y_test = train_test_split(data_new.iloc[:,:], data['genre'], test_size = 0.2, random_state=1)

In [ ]:
print(X_train.info())

In [ ]:
# Explore test set
print(X_test.info())

**Standardize the data**

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#SelectKBest(f_classif, k=20).fit_transform(X_train, Y_train)
f_classif(X_train_scaled, Y_train)
np.mean(f_classif(X_train_scaled, Y_train)[0])

**Initial random forest model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf=RandomForestClassifier(n_estimators=200)
clf.fit(X_train_scaled,Y_train)

In [ ]:
Y_pred=clf.predict(X_test_scaled)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

**Cross validation**

In [ ]:
cv_scores_dict = {}
criterion_list = ['gini', 'entropy']
for n_estimators in [200, 300,400,500]:
    cv_scores = []
    for criterion in criterion_list:    
        RM_model = RandomForestClassifier(n_estimators = n_estimators, criterion=criterion,random_state=1)
        cv_scores.append(np.mean(cross_val_score(RM_model, X_train_scaled, Y_train, cv=10,n_jobs=-1)))
    max_cv_score = max(cv_scores)
    max_criterion = criterion_list[cv_scores.index(max_cv_score)]
    cv_scores_dict[n_estimators] = {'criterion': max_criterion, 'cv_score': max_cv_score, 'n_estimators': n_estimators}
print(cv_scores_dict)

**Random forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf=RandomForestClassifier(n_estimators=400,criterion='gini' ,random_state=1)
clf.fit(X_train_scaled,Y_train)

In [ ]:
Y_pred=clf.predict(X_test_scaled)

In [ ]:
conf_matrix = metrics.confusion_matrix(Y_test,Y_pred)
sns.heatmap(conf_matrix, annot = True, fmt = ".2f", square = True, cmap = plt.cm.Blues)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion matrix')
plt.tight_layout()
print(conf_matrix)

In [ ]:
accuracy = metrics.accuracy_score(Y_test, Y_pred)
error = 1 - accuracy
precision = metrics.precision_score(Y_test, Y_pred, average = None)
recall = metrics.recall_score(Y_test, Y_pred, average = None)
F1_score = metrics.f1_score(Y_test, Y_pred, average = None)
print("Accuracy",accuracy,"\nError", error,"\nPrecision", precision,"\nRecall", recall,"\nF1_score", F1_score)

**Parameter variations**

In [ ]:
sample_size = [1000,2000,3000,4000,5000,6000]
accuracy=[]
for i in sample_size:
  clf=RandomForestClassifier(n_estimators=400,criterion='gini' ,random_state=1,max_samples=i)
  clf.fit(X_train_scaled,Y_train)
  Y_pred=clf.predict(X_test_scaled)
  accuracy.append(metrics.accuracy_score(Y_test, Y_pred))
  print("Sample size: ",i," Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

plt.plot(sample_size, accuracy)
  

In [ ]:
max_leaf_nodes = [200,400,600,800,1000,1200]
accuracy=[]
for i in max_leaf_nodes:
  clf=RandomForestClassifier(n_estimators=400,criterion='gini' ,random_state=1,max_leaf_nodes=i)
  clf.fit(X_train_scaled,Y_train)
  Y_pred=clf.predict(X_test_scaled)
  accuracy.append(metrics.accuracy_score(Y_test, Y_pred))
  print("Max Leaf Node: ",i," Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

plt.plot(max_leaf_nodes, accuracy)
  

In [ ]:
max_features = [10,20,30,40,50,60,70]
accuracy=[]
for i in max_features:
  clf=RandomForestClassifier(n_estimators=400,criterion='gini' ,random_state=1,max_features=i)
  clf.fit(X_train_scaled,Y_train)
  Y_pred=clf.predict(X_test_scaled)
  accuracy.append(metrics.accuracy_score(Y_test, Y_pred))
  print("Max Features: ",i," Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

plt.plot(max_features, accuracy)

**Top Features**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

# Build a forest and compute the impurity-based feature importances
forest = clf=RandomForestClassifier(n_estimators=400,criterion='gini',random_state=1)

forest.fit(X_train_scaled, Y_train)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train_scaled.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train_scaled.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train_scaled.shape[1]), indices)
plt.xlim([-1, X_train_scaled.shape[1]])
plt.show()